<a href="https://colab.research.google.com/github/mmistroni/jupyter/blob/master/PySpark_Colab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Installing dependencies</h3>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType, LongType
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType



spark = SparkSession.builder.master("local[*]").getOrCreate()

<p> ToDO test few runs here, then  store a dataframe of all possible shares for last 2 years and run a correlation between all cols </b>

In [ ]:
simple_df = \
"""dept,cluster,usd_amt,cluster_amt
D1,,10,50
D2,C2,20,100
D3,C2,70,100
D1,C2,10,100
D5,C1,40,50
D3,C3,70,120
D4,C3,50,120"""
data = simple_df.split('\n')
rows = [ln.split(',') for ln in data]
headers = rows[0]
vals = rows[1:] 
rdd = spark.sparkContext.parallelize(vals)
schema = StructType([StructField(fname, StringType(), True) for fname in headers])
df = spark.createDataFrame(rdd, schema)







In [ ]:
df.orderBy(*['dept', 'cluster']).toPandas()

In [ ]:
tt = df.groupby('dept').agg(F.sum('cluster_amt').alias('total_clust_amt'))

In [ ]:
tt.toPandas()

In [ ]:
jnd = tt.join(df, ['dept'])
no_func_res = jnd.withColumn('dept_pcnt', F.col('cluster_amt')/F.col('total_clust_amt')).orderBy(*['dept','cluster'])
no_func_res.show()

In [ ]:
# TEstign window function on df
df.show()

In [ ]:
simple_df = \
"""le,cpty,cpty_cs_or_le
CSI,First,3rd Bank
CSI,First,3rd Investment
CSI,First,3rd Bank
CSI,First,3rd PincoPallino
CSI,First,3rd Bank
CSI,First,3rd Bank"""
data = simple_df.split('\n')
rows = [ln.split(',') for ln in data]
headers = rows[0]
vals = rows[1:] 
rdd = spark.sparkContext.parallelize(vals)
schema = StructType([StructField(fname, StringType(), True) for fname in headers])
df = spark.createDataFrame(rdd, schema)
df.show()


In [ ]:
from pyspark.sql.window import Window
w2 = Window.partitionBy('le', 'cpty')
res = df.withColumn('tparty_cs_or_le',F.collect_set('cpty_cs_or_le').over(w2))
#concat_ws(',', split(df.emailed))
res = res.withColumn('concatenated_le', F.concat_ws(',', 'tparty_cs_or_le'))
res.select('concatenated_le').collect()

In [ ]:
res.schema

In [ ]:
empsalary = [
  ["sales",     1,  "Alice",  5000],
  ["personnel", 2,  "Olivia", 3900],
  ["sales",     3,  "Ella",   4800],
  ["sales",     4,  "Ebba",   4800],
  ["personnel", 5,  "Lilly",  3500],
  ["develop",   7,  "Astrid", 4200],
  ["develop",   8,  "Saga",   6000],
  ["develop",   9,  "Freja",  4500],
  ["develop",   10, "Wilma",  5200],
  ["develop",   11, "Maja",   5200]]

rdd = spark.sparkContext.parallelize(empsalary)
schema = StructType([StructField('depName', StringType(), True),
                     StructField('empNo', LongType(), True),
                     StructField('name', StringType(), True) ,
                     StructField('salary', LongType(), True)])
empsalary_df = spark.createDataFrame(rdd, schema)




In [ ]:
from pyspark.sql.window import Window
w = Window.partitionBy('depName')


In [ ]:
empsalary_df.withColumn('total_rank',F.sum('salary').over(w)).show()

In [ ]:




import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

def calculate_month_diffs(maxdate, current):
  pass


with_dt = df.withColumn('cobdate_str', F.concat(F.col('year'),F.lit('-'), F.col('month'))) # this is the first dataframe
print('with dt has:{}'.format(with_dt.count()))
with_cobdt = with_dt.select('cobdate_str').distinct().withColumn('cobdate', F.to_date(F.col('cobdate_str'))).distinct()
print('Wtih cobdt has:{}'.format(with_cobdt.count()))

max_dt = with_cobdt.select(F.max(F.col('cobdate')).alias('max_date')).collect()[0]['max_date']
with_mx = with_cobdt.withColumn('max_dt', F.lit(max_dt))
res = with_mx.withColumn('month_diffs', F.months_between(F.col('max_dt'), F.col('cobdate')).cast(IntegerType()))\
            .filter(F.col('month_diffs') <= 12)

joined = with_dt.join(res, ['cobdate_str'])

joined.toPandas()



In [ ]:
from datetime import datetime
from dateutil import relativedelta

##Aug 7 1989 8:10 pm
date_1 = datetime(1989, 8, 1, 20, 10)

##Dec 5 1990 5:20 am
date_2 = datetime(1990, 12, 5, 5, 20)

#This will find the difference between the two dates
difference = relativedelta.relativedelta(date_2, date_1)

years = difference.years
months = difference.months
days = difference.days
hours = difference.hours
minutes = difference.minutes

print ("Difference is %s year, %s months, %s days, %s hours, %s minutes " %(years, months, days, hours, minutes))

In [ ]:
df.toPandas()



In [ ]:
# Use this link to calculate correlation https://stackoverflow.com/questions/45112976/how-to-use-correlation-in-spark-with-dataframes

In [ ]:
!pip install pandas-datareader

<h3>Getting IEXApi Token </h3>

In [ ]:
def get_iexapi_keys():
  from google.colab import drive
  drive.mount('/content/gdrive')
  with open('gdrive/My Drive/passwords/iexapi.keys') as f:
    return f.readlines()[0]


In [ ]:
import requests
import pandas_datareader.data as dr
import pandas as pd
from datetime import date

iexapi_token = get_iexapi_keys()


def get_historical_value(symbol):
  try: 
    data = dr.get_data_yahoo(symbol, date(2018,1,1), date(2019,9,19))[['Adj Close']]
    return data.rename(columns={'Adj Close' : symbol})
    
  except Exception as e :
    print('Exception for {}={}'.format(symbol, str(e)))
    return pd.DataFrame(columns=[symbol])
  
def get_all_symbols():
  all_symbols_data = requests.get('https://cloud.iexapis.com/stable/ref-data/iex/symbols?token={}'.format(iexapi_token)).json()
  good_ones = [d['symbol'] for d in all_symbols_data if d['isEnabled']]
  return filter(lambda data: bool(data), good_ones)

def get_all_etfs():
  nyse_symbols = requests.get('https://cloud.iexapis.com/stable/ref-data/exchange/nys/symbols?token={}'.format(iexapi_token)).json()
  nas_symbols = requests.get('https://cloud.iexapis.com/stable/ref-data/exchange/nas/symbols?token={}'.format(iexapi_token)).json()
  return [d['symbol'] for d in nyse_symbols + nas_symbols if d['type'].lower() == 'et']

def get_all_stocks_data():
  all_symbols_data =  get_all_etfs()
  return map(lambda symbol: get_historical_value(symbol), all_symbols_data)


<h3> Comparing shares correlation with VIX via Pandas and Spark </h3>

In [ ]:
vix = get_historical_value('^VIX')


In [ ]:

all_stocks  = get_all_stocks_data()

res= [vals for vals in all_stocks if vals.shape[0] == vix.shape[0]]
res.append(vix)



In [ ]:
# building pandas df
all_data = pd.concat(res, axis=1)
spark_df = spark.createDataFrame(all_data)



In [ ]:
def calculate_correlation(df):
  from pyspark.ml.stat import Correlation
  from pyspark.ml.feature import VectorAssembler

  # convert to vector column first
  vector_col = "corr_features"
  assembler = VectorAssembler(inputCols=df.columns, outputCol=vector_col)
  df_vector = assembler.transform(df).select(vector_col)

  # get correlation matrix
  matrix = Correlation.corr(df_vector, vector_col)
  return matrix


In [ ]:
columns = spark_df.columns
matrix = calculate_correlation(spark_df)
matrix.toPandas()


In [ ]:
# convert to numpy
np_array = matrix.collect()[0]["pearson({})".format("corr_features")].toArray()
np_array.shape

vix_row = np_array[-1:].tolist()[0]


In [ ]:
zipped = zip(columns, vix_row)

sorted_zip = sorted(zipped, key=lambda tpl:tpl[1], reverse=True)
from pprint import pprint
pprint(list(sorted_zip)[0:10])


In [ ]:
df = spark.createDataFrame(data, ["features"])

r1 = Correlation.corr(df, "features").head()
print("Pearson correlation matrix:\n" + str(r1[0]))

<h3> Alternative approach via RDD </h3>

In [ ]:
def get_historical_value_df(symbol):
  try: 
    data = dr.get_data_yahoo(symbol, date(2018,1,1), date(2019,9,19))[['Adj Close']]
    pandas_df = data.rename(columns={'Adj Close' : 'adj_close'})['adj_close']
    return spark.createDataFrame(pandas_df, FloatType())
    
  except Exception as e :
    print('Excepiton for {}:{}'.format(symbol, str(e)))
    return []

def calculate_correlation(df1, df2, field1, field2)
  firstRDD = df1.select("adj_close").map(lambda r: row.getDouble(0))
  val secondRDD: RDD[Double] = yourDF.select("field2").map(row => row.getDouble(0))
  val corr = Statistics.corr(firstRDD, secondRDD, "spearman")  
  

In [ ]:
vix = get_historical_value_df('^VIX')

vals = [x for x in map(lambda r: r.asDict(), vix)]
from pprint import pprint
vals






<h3> Testing date selection </h3>

In [ ]:
ranges = list(range(0,13))
list_2018 = [2018] * 13
zipped1 = list(zip(list_2018, ranges))
zipped2 = list(zip([2019] * 13, ranges))
zipped3 = list(zip([2020] * 1, ranges))

zipped1 + zipped2 + zipped3

In [ ]:
from datetime import date
import datetime
import pandas as pd
def create_filter(current_dt):
  # we take everything for current year
  current_month = current_dt.month
  current_year = current_dt.year
  print('Current month:{}, Current year:{}'.format(current_month, current_year))
  curent_yr_filter = (F.col('year') == current_year) & (F.col('month') <= current_month)
  print('currnet year filter is:{}'.format(curent_yr_filter)) 
  last_yr_start = (pd.to_datetime(current_dt) - pd.DateOffset(months=12))
  print('las tyer starting from:{}'.format(last_yr_start))
  last_yr_month = last_yr_start.month
  print('Including zeros..')
  real_month = last_yr_month if last_yr_month > 1 else 0
  last_yr_filter = (F.col('year') == current_year-1) & (F.col('month') >= real_month)
  print('Last year filter:{}'.format(last_yr_filter))
  

  



In [ ]:
for dt in [date(2020,1,1),
           date(2019,12,31),
           date(2019,11,30),
           date(2019,10,31)]:
  print('TEsting:{}'.format(dt))
  create_filter(dt)
  print('-----')


In [ ]:
import datetime
import pandas as pd
(pd.to_datetime(test_dt) - pd.DateOffset(months=12)).month